## Download Data from Google-Drive

In [1]:
'''
File Name : Data
File Link : https://drive.google.com/file/d/1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw/view?usp=share_link
File Id : '1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw'

'''
!gdown --id 1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw
!unzip ccle_ctrpv2_gdse.zip

/bin/bash: /home/amirhosein/anaconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/home/amirhosein/anaconda3/envs/tf/lib/python3.9/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw
From (redirected): https://drive.google.com/uc?id=1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw&confirm=t&uuid=79f5c2c9-1ad3-4b7f-9920-2288744e374d
To: /home/amirhosein/Documents/Codes/Drug Response Prediction/ccle_ctrpv2_gdse.zip
 10%|███▉                                   | 57.1M/557M [00:19<03:51, 2.16MB/s]^C
Traceback (most recent call last):
  File "/home/amirhosein/anaconda3/envs/tf/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/home/amirhosein/anaconda3/envs/tf/lib/python3.9/site-packages/gdown/cli.py", line 156, in main
    f

## Import necessary libraries

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error , mean_absolute_error
from time import perf_counter, sleep
import math
import scipy.stats as stats
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

## Read train and test data

In [5]:
# x_train
ccle_ctrpv2 = pd.read_csv("../data/ccle_ctrpv2_rnaseq_tpm.csv")
# x_test
ccle_ctrpv2_aac = pd.read_csv("../data/ccle_ctrpv2_aac.csv")
# y_train
gdse_rnaseq = pd.read_csv("../data/gdse_rnaseq_tpm.csv")
# y_test
gdse_aac = pd.read_csv("../data/gdse_aac.csv")

In [6]:

def get_top_500cor(X_train_normalized, Y_train, columns):
    corrs =  np.zeros(X_train_normalized.shape[1])
    for j in range(X_train_normalized.shape[1]):
        corrs[i], _ = pearsonr(X_train_normalized[:,j], Y_train)

    sorted_indices = np.argsort(corrs)[::-1]
    feature_names = columns
    sorted_names = [feature_names[i] for i in sorted_indices]
    
    return sorted_names[:500]

## Cleaning the dataframes

In [7]:
drug_names = gdse_aac.columns.values.tolist()
drug_names = drug_names[1:]


ccle_ctrpv2 = ccle_ctrpv2.rename(columns={"Unnamed: 0": "sample"})
gdse_rnaseq = gdse_rnaseq.rename(columns={"Unnamed: 0": "sample"})
ccle_ctrpv2_aac = ccle_ctrpv2_aac.rename(columns={"Unnamed: 0": "sample"})
gdse_aac = gdse_aac.rename(columns={"Unnamed: 0": "sample"})

In [8]:
def remove_nan(data):
    data = data.dropna()
    data.reset_index(drop = True)
    return data

In [9]:
'''
Initializing k-fold cross-validation.

n_splits=4: the dataset will be split into 4 equal-sized folds or subsets.

shuffle: data will be shuffled before splitting into folds, 
         ensuring randomization in the distribution of samples across each fold.

'''
kfold = KFold(n_splits=4, shuffle=True)


'''
Initializing StandardScaler to normalize the data to have mean of 0 and variance of 1.
'''
Sscaler = StandardScaler()

## Creating a shallow neural network
---
1. **Layers**

First Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>
Second Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>
Third Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>
Fourth Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>

Output Layer -> **Activation Function:** *Sigmoid* <br>

Sigmoid activation function is used to force the output to have a range between 0 and 1, 
as the desired output (Drug Response) has a value between 0 and 1.

2. **Optimizer**

Using *Stochastic Gradient descent* (SGD) with momentum:

        learning rate: 0.01
        momentum = 0.9

3. **Loss Function**

Using *mean squared error* (MSE) for the loss function

4. **Regularization technique**

20% dropout after each hidden layer 

5. **Network schematic**

![MarineGEO circle logo](Arch.png)

In [10]:
model = tf.keras.models.Sequential()

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1 , activation='sigmoid'))


learning_rate = 0.01
momentum = 0.9
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate, momentum)

optimizer_rmsprop = tf.keras.optimizers.RMSprop()

# Training procedure

*For each drug in the drug list:*
   1. Split the data in to train and test sets, and then remove the Nan values.
   2. Merging X_train with ytrain, and X_test with  y_test because the samples in each are different.
   3. Normalize the training X. </br>
   
   *Running the following 6 times:*
   
       4. Applying 4-fold Cross Valisation
       5. Applying the relevant features selection technique, and extract the selected features.
       6. Train the model based on the CV data and extracted features
       7. Saving the best model performed in CV and the best features
       
   8. Predicting the test set using the best model from the previous section
   9. Calculating the correlation score between the predicted drug_respons and the actual drug_response

In [11]:
cor_df = pd.DataFrame([], columns = ['corr'])

correlatoins = []
correlatoins = []
mse_train_list = []
rmse_train_list = []
mae_train_list = []
spearmanr_correlation_train_list = []
kendalltau_correlation_train_list = []
cr = []
mse = []
rmse = []
mae = []
spearmanr =[]
kendalltau = []
start = perf_counter()
for idx, drug in enumerate(drug_names):
    
    selected_features = []
    best_model_top_features = set()
    model_performances = []
    model_acc = []
    pearson_correlations = []
    saved_features = []

    '''
        train test split
    '''
    y_train = ccle_ctrpv2_aac[['sample', drug]]
    y_test  = gdse_aac[['sample', drug]]

    y_train = remove_nan(y_train)
    y_test  = remove_nan(y_test)

    ccle_ctrpv2 = remove_nan(ccle_ctrpv2)
    gdse_rnaseq = remove_nan(gdse_rnaseq)


    '''
    Merge
    '''

    gdse_rnaseq_merged_df = pd.merge(gdse_rnaseq, y_test, on='sample', how='inner')
    ccle_ctrpv2_merged_df = pd.merge(ccle_ctrpv2, y_train, on='sample', how='inner')

    X_train = ccle_ctrpv2_merged_df.iloc[:, 1: len(gdse_rnaseq.columns)]
    X_test  = gdse_rnaseq_merged_df.iloc[:, 1: len(gdse_rnaseq.columns)]

    Y_train = ccle_ctrpv2_merged_df.iloc[:, len(gdse_rnaseq.columns):]
    Y_test  = gdse_rnaseq_merged_df.iloc[:, len(gdse_rnaseq.columns):]

    X_train_norm = Sscaler.fit_transform(X_train)
    X_train_normalized = pd.DataFrame(X_train_norm, columns=X_train.columns)
    
    for i in range(6):
            
        selected_features = []
        best_model_top_features = set()
        model_performances = []
        model_acc = []
        pearson_correlations = []
        best_model_path = 'saved_models/best_model_'+ drug + str(i)
        saved_features = []
        
        for train, validation in kfold.split(X_train_normalized , Y_train):

            train_data = X_train_normalized.iloc[train]
            train_labels = Y_train.iloc[train]
            val_data = X_train_normalized.iloc[validation]
            val_labels = Y_train.iloc[validation]
            
            
            top_features = get_top_500cor(np.asarray(train_data), train_labels, train_data.columns)
            
            
            train_data_selected = train_data[top_features]
            val_data_selected = val_data[top_features]


            model.compile(optimizer= optimizer_sgd, loss='mean_squared_error', metrics='MAE')

            checkpoint_callback = ModelCheckpoint(filepath=best_model_path+ '.h5' , monitor='MAE', save_best_only=True, mode='min')

            history = model.fit(train_data_selected, train_labels, epochs=15,callbacks=[checkpoint_callback], validation_data=(val_data_selected, val_labels))
            saved_features.append({
                'top_features': top_features,
                'model_performances': history.history['MAE'][-1]
            })

        saved_features = sorted(saved_features, key=lambda x: x['model_performances'])
        top_features_saved = saved_features[0].get('top_features')
        top_features_saved = np.asarray(top_features_saved)
        new_X = X_test[top_features_saved]

        Xtest_norm = Sscaler.fit_transform(new_X)
        Xtest_normalized = pd.DataFrame(Xtest_norm, columns=new_X.columns)

        loaded_model = tf.keras.models.load_model(best_model_path+ '.h5' )

        gdsc_predictions = loaded_model.predict(Xtest_normalized)

        Y_test_array = np.asarray(Y_test)

        correlation, _ = pearsonr(Y_test_array.flatten(),  gdsc_predictions.flatten())
        mse_train = mean_squared_error(Y_test_array.flatten(), gdsc_predictions.flatten())
        rmse_train = math.sqrt(mse_train)
        mae_train = mean_absolute_error(Y_test_array.flatten(), gdsc_predictions.flatten())

    # Calculate Correlation for the validation data
        spearmanr_correlation_train, _ = stats.spearmanr(Y_test_array.flatten(), gdsc_predictions.flatten())
        kendalltau_correlation_train, _ = stats.kendalltau(Y_test_array.flatten(), gdsc_predictions.flatten())
    
        correlatoins.append(correlation)
        mse_train_list.append(mse_train)
        rmse_train_list.append(rmse_train)
        mae_train_list.append(mae_train)
        spearmanr_correlation_train_list.append(spearmanr_correlation_train)
        kendalltau_correlation_train_list.append(kendalltau_correlation_train)

    index = correlatoins.index(max(correlatoins))
    cr.append(max(correlatoins))
    mse.append(mse_train_list[index])
    rmse.append(rmse_train_list[index])
    mae.append(mae_train_list[index])
    spearmanr.append(spearmanr_correlation_train_list[index])
    kendalltau.append(kendalltau_correlation_train_list[index])
    correlatoins = []
    mse_train_list = []
    rmse_train_list = []
    mae_train_list = []
    spearmanr_correlation_train_list = []
    kendalltau_correlation_train_list = []
    
end = perf_counter()

Epoch 1/15
18/18 [==============================] - 1s 18ms/step - loss: 0.0422 - MAE: 0.1642 - val_loss: 0.0122 - val_MAE: 0.0906
Epoch 2/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0185 - MAE: 0.1115 - val_loss: 0.0106 - val_MAE: 0.0845
Epoch 3/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0144 - MAE: 0.0975 - val_loss: 0.0087 - val_MAE: 0.0749
Epoch 4/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0127 - MAE: 0.0894 - val_loss: 0.0087 - val_MAE: 0.0740
Epoch 5/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0119 - MAE: 0.0884 - val_loss: 0.0085 - val_MAE: 0.0734
Epoch 6/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0113 - MAE: 0.0838 - val_loss: 0.0082 - val_MAE: 0.0723
Epoch 7/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0103 - MAE: 0.0797 - val_loss: 0.0081 - val_MAE: 0.0719
Epoch 8/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0103 - MAE: 0.0

18/18 [==============================] - 0s 6ms/step - loss: 0.0073 - MAE: 0.0681 - val_loss: 0.0082 - val_MAE: 0.0711
Epoch 4/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0069 - MAE: 0.0663 - val_loss: 0.0083 - val_MAE: 0.0713
Epoch 5/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0073 - MAE: 0.0679 - val_loss: 0.0081 - val_MAE: 0.0704
Epoch 6/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0072 - MAE: 0.0679 - val_loss: 0.0079 - val_MAE: 0.0694
Epoch 7/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0067 - MAE: 0.0648 - val_loss: 0.0079 - val_MAE: 0.0692
Epoch 8/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0064 - MAE: 0.0631 - val_loss: 0.0078 - val_MAE: 0.0689
Epoch 9/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0066 - MAE: 0.0648 - val_loss: 0.0077 - val_MAE: 0.0682
Epoch 10/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0066 - MAE: 0.0645 - val_l

18/18 [==============================] - 0s 4ms/step - loss: 0.0052 - MAE: 0.0551 - val_loss: 0.0040 - val_MAE: 0.0492
Epoch 6/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0049 - MAE: 0.0528 - val_loss: 0.0040 - val_MAE: 0.0489
Epoch 7/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0048 - MAE: 0.0527 - val_loss: 0.0039 - val_MAE: 0.0488
Epoch 8/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0049 - MAE: 0.0529 - val_loss: 0.0039 - val_MAE: 0.0487
Epoch 9/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0050 - MAE: 0.0538 - val_loss: 0.0039 - val_MAE: 0.0489
Epoch 10/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0048 - MAE: 0.0535 - val_loss: 0.0039 - val_MAE: 0.0486
Epoch 11/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0051 - MAE: 0.0553 - val_loss: 0.0039 - val_MAE: 0.0484
Epoch 12/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0046 - MAE: 0.0523 - val

18/18 [==============================] - 0s 6ms/step - loss: 0.0040 - MAE: 0.0483 - val_loss: 0.0036 - val_MAE: 0.0452
Epoch 8/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0039 - MAE: 0.0484 - val_loss: 0.0036 - val_MAE: 0.0456
Epoch 9/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0039 - MAE: 0.0483 - val_loss: 0.0036 - val_MAE: 0.0458
Epoch 10/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0039 - MAE: 0.0491 - val_loss: 0.0036 - val_MAE: 0.0454
Epoch 11/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0039 - MAE: 0.0485 - val_loss: 0.0036 - val_MAE: 0.0456
Epoch 12/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0040 - MAE: 0.0484 - val_loss: 0.0037 - val_MAE: 0.0460
Epoch 13/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0038 - MAE: 0.0466 - val_loss: 0.0036 - val_MAE: 0.0459
Epoch 14/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0039 - MAE: 0.0486 - v

18/18 [==============================] - 0s 4ms/step - loss: 0.0032 - MAE: 0.0442 - val_loss: 0.0047 - val_MAE: 0.0512
Epoch 10/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0034 - MAE: 0.0451 - val_loss: 0.0047 - val_MAE: 0.0512
Epoch 11/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0037 - MAE: 0.0475 - val_loss: 0.0048 - val_MAE: 0.0515
Epoch 12/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0033 - MAE: 0.0450 - val_loss: 0.0049 - val_MAE: 0.0520
Epoch 13/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0036 - MAE: 0.0463 - val_loss: 0.0048 - val_MAE: 0.0518
Epoch 14/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0033 - MAE: 0.0448 - val_loss: 0.0047 - val_MAE: 0.0510
Epoch 15/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0034 - MAE: 0.0463 - val_loss: 0.0047 - val_MAE: 0.0508
Epoch 1/15
18/18 [==============================] - 1s 15ms/step - loss: 0.0038 - MAE: 0.0472 -

...

17/17 [==============================] - 0s 8ms/step - loss: 0.0064 - MAE: 0.0636 - val_loss: 0.0069 - val_MAE: 0.0677
Epoch 15/15
17/17 [==============================] - 0s 9ms/step - loss: 0.0062 - MAE: 0.0618 - val_loss: 0.0068 - val_MAE: 0.0672
Epoch 1/15
17/17 [==============================] - 1s 18ms/step - loss: 0.0069 - MAE: 0.0661 - val_loss: 0.0059 - val_MAE: 0.0596
Epoch 2/15
17/17 [==============================] - 0s 9ms/step - loss: 0.0063 - MAE: 0.0634 - val_loss: 0.0059 - val_MAE: 0.0595
Epoch 3/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0065 - MAE: 0.0635 - val_loss: 0.0059 - val_MAE: 0.0596
Epoch 4/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0066 - MAE: 0.0637 - val_loss: 0.0059 - val_MAE: 0.0598
Epoch 5/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0065 - MAE: 0.0636 - val_loss: 0.0060 - val_MAE: 0.0601
Epoch 6/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0066 - MAE: 0.0641 - val_

17/17 [==============================] - 1s 20ms/step - loss: 0.0051 - MAE: 0.0555 - val_loss: 0.0039 - val_MAE: 0.0500
Epoch 2/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0055 - MAE: 0.0582 - val_loss: 0.0039 - val_MAE: 0.0501
Epoch 3/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0053 - MAE: 0.0575 - val_loss: 0.0040 - val_MAE: 0.0503
Epoch 4/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0050 - MAE: 0.0559 - val_loss: 0.0039 - val_MAE: 0.0500
Epoch 5/15
17/17 [==============================] - 0s 9ms/step - loss: 0.0050 - MAE: 0.0550 - val_loss: 0.0039 - val_MAE: 0.0498
Epoch 6/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0051 - MAE: 0.0553 - val_loss: 0.0038 - val_MAE: 0.0494
Epoch 7/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0050 - MAE: 0.0555 - val_loss: 0.0038 - val_MAE: 0.0498
Epoch 8/15
17/17 [==============================] - 0s 8ms/step - loss: 0.0048 - MAE: 0.0539 - val_l

17/17 [==============================] - 0s 7ms/step - loss: 0.0038 - MAE: 0.0490 - val_loss: 0.0032 - val_MAE: 0.0449
Epoch 4/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0040 - MAE: 0.0496 - val_loss: 0.0032 - val_MAE: 0.0453
Epoch 5/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0038 - MAE: 0.0490 - val_loss: 0.0032 - val_MAE: 0.0452
Epoch 6/15
17/17 [==============================] - 0s 8ms/step - loss: 0.0037 - MAE: 0.0486 - val_loss: 0.0032 - val_MAE: 0.0449
Epoch 7/15
17/17 [==============================] - 0s 8ms/step - loss: 0.0036 - MAE: 0.0462 - val_loss: 0.0031 - val_MAE: 0.0445
Epoch 8/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0037 - MAE: 0.0472 - val_loss: 0.0032 - val_MAE: 0.0453
Epoch 9/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0037 - MAE: 0.0483 - val_loss: 0.0033 - val_MAE: 0.0456
Epoch 10/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0039 - MAE: 0.0485 - val_l

17/17 [==============================] - 0s 7ms/step - loss: 0.0028 - MAE: 0.0423 - val_loss: 0.0031 - val_MAE: 0.0440
Epoch 6/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0028 - MAE: 0.0420 - val_loss: 0.0030 - val_MAE: 0.0439
Epoch 7/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0029 - MAE: 0.0421 - val_loss: 0.0031 - val_MAE: 0.0445
Epoch 8/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0027 - MAE: 0.0417 - val_loss: 0.0031 - val_MAE: 0.0447
Epoch 9/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0028 - MAE: 0.0419 - val_loss: 0.0031 - val_MAE: 0.0446
Epoch 10/15
17/17 [==============================] - 0s 9ms/step - loss: 0.0028 - MAE: 0.0415 - val_loss: 0.0031 - val_MAE: 0.0443
Epoch 11/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0028 - MAE: 0.0423 - val_loss: 0.0031 - val_MAE: 0.0445
Epoch 12/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0030 - MAE: 0.0437 - val

17/17 [==============================] - 0s 6ms/step - loss: 0.0025 - MAE: 0.0397 - val_loss: 0.0018 - val_MAE: 0.0330
Epoch 8/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0024 - MAE: 0.0394 - val_loss: 0.0018 - val_MAE: 0.0328
Epoch 9/15
17/17 [==============================] - 0s 9ms/step - loss: 0.0024 - MAE: 0.0389 - val_loss: 0.0019 - val_MAE: 0.0332
Epoch 10/15
17/17 [==============================] - 0s 9ms/step - loss: 0.0024 - MAE: 0.0387 - val_loss: 0.0019 - val_MAE: 0.0335
Epoch 11/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0024 - MAE: 0.0390 - val_loss: 0.0019 - val_MAE: 0.0336
Epoch 12/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0025 - MAE: 0.0397 - val_loss: 0.0019 - val_MAE: 0.0338
Epoch 13/15
17/17 [==============================] - 0s 9ms/step - loss: 0.0024 - MAE: 0.0386 - val_loss: 0.0019 - val_MAE: 0.0340
Epoch 14/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0024 - MAE: 0.0392 - v

# Saving the Results

In [12]:
execution_time = end - start

df = pd.DataFrame({"Drugs": drug_names, "pearsonCor": cr , "spearmanCor": spearmanr , "kendallCor": kendalltau , 
                  "RMSE": rmse ,  "MSE": mse , "MAE": mae , "Time": execution_time})
df.to_csv('DL-results/Result_cor5000.csv', index=False)

